# Instacart Capstone Project

In [1]:
# Importing the libraries and datasets
import pandas as pd

aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
# order_products_prior = pd.read_csv('order_products_prior.csv')
order_products_train = pd.read_csv('order_products_train.csv')
orders = pd.read_csv('orders.csv')
products = pd.read_csv('products.csv')

In [2]:
# Indexing the aisles and departments data
departments.index = departments.department_id
departments = departments.department
aisles.index = aisles.aisle_id
aisles = aisles.aisle

In [3]:
# To create a usable base dataframe without IDs
base_products = products.join(aisles, on = 'aisle_id', how = 'right')
base_products = base_products.join(departments, on = 'department_id', how = 'right')
base_products.index = base_products.product_id
base_products = base_products[['product_name', 'aisle', 'department']].sort_index()
base_products['organic'] = (base_products['product_name'].str.contains('Organic')).astype(int)

In [8]:
# To determine how many organic products there are
print "The total products are %s" % (len(base_products))
organic_products = base_products[base_products['product_name'].str.contains('Organic')]
print "The total organic products are %s" % (len(organic_products))

The total products are 49688
The total organic products are 5035


In [12]:
# Selecting the train and test dataframe
order_num = order_products_train.order_id.unique()
order_num = pd.DataFrame(order_num)
order_num_exp = order_num.sample(n=1000).sort_index()
order_num_exp['order_id'] = order_num_exp.astype(int)
order_num_exp['in_test'] = 1
order_num_exp.index = order_num_exp.order_id

order_products_train.index = order_products_train.order_id

order_num_exp = order_num_exp.in_test
order_products_exp = order_products_train.join(order_num_exp, on = 'order_id', how = 'right')
order_products_exp = order_products_exp[['order_id', 'product_id']]
order_products_exp = order_products_exp.join(base_products, on = 'product_id', how = 'left')

order_products_exp = order_products_exp.sort_index()
order_products_exp = order_products_exp[['product_name', 'aisle', 'department', 'organic']]

# must reset the index to parse through data
order_products_exp_test = order_products_exp.reset_index(drop = True)

print len(order_products_exp)

10179


In [13]:
# Testing the algorithm for aisles
# Cleaning the dataframe
import re
import nltk
nltk.download('stopwords') # list of words that are generally irrelevant # just downloads the list
from nltk.corpus import stopwords # importing the list of stopwords
from nltk.stem.porter import PorterStemmer # importing module to stem only the root words
corpus = [] # collection of texts
for i in range(0,10179): # to loop through all of the rows of the dataframe
    review = re.sub('[^a-zA-Z]', ' ', order_products_exp_test['aisle'][i]) # to only keep the letters # need the i to parse through every index
    review = review.lower() # to make all of the letters lowercase
    review = review.split() # to split the review from string into list of different words
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))] # removing all english words that are in the stopwords list # changing for only the stem/root words
    review = ' '.join(review) # to join the list of words back into a string
    corpus.append(review) # to add to the "collection of texts"

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timothyyamaguchi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() # caps the limit of words removing the "non relevant" words
X = cv.fit_transform(corpus).toarray() # creating a sparse matrix # like a matrics of features
y = order_products_exp.iloc[:, 3].values # to take the results of the reviews 

# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

print cm

[[ 297 1139]
 [   7  593]]


In [34]:
percentage_correct = (297.+593)/(297+593+1139+7)*100
print "The model correctly analyzed %s percent correct." % (percentage_correct)

The model correctly analyzed 43.7131630648 percent correct.
